In [5]:
# Importo le librerie necessarie per il codice

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

# Elenco dei file da leggere
files = ['portieri_23.xlsx', 'portieri_22.xlsx', 'portieri_21.xlsx', 'portieri_20.xlsx', 'portieri_19.xlsx']

# Faccio un dizionario per mappare i file agli anni
year_mapping = {'portieri_23.xlsx': 2023,'portieri_22.xlsx': 2022,'portieri_21.xlsx': 2021,'portieri_20.xlsx': 2020,'portieri_19.xlsx': 2019}



In [7]:
# Questa parte di codice crea 5 figure che rappresentano il riquadro di correlazione fra le statistiche
# suddiviso per annata

stat1 = 'Saves'
stat2 = 'Age'
correlations = []
years = []

for file in files:
    # Leggi il file Excel
    data_frame = pd.read_excel(file)

    # Uso .drop per rimuovere dal data frame quelle statistiche che non voglio usare in questa analisi
    data_frame.drop(columns=['Player', 'Nation', 'Pos', 'Squad'], inplace=True)

    # Calcolo la matrice di correlazione
    corr_matrix = data_frame.corr()

    # Ottengo l'anno dal file usando il mapping
    year = year_mapping[file]
    years.append(year)

    # Ottengo in output il grafico di correlazione
    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")
    fig.show()

    # Questa parte di codice serve per conoscere la correlazione di due statistiche specificate, annata per annata
    if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
        correlation_value = corr_matrix.loc[stat1, stat2]
        correlations.append(correlation_value)
    else:
        print(f"Anno {year}: Una o entrambe le statistiche non sono presenti nel dataset.")
        correlations.append(None)  # Aggiunge un valore nullo se manca la correlazione

# Crea un DataFrame con le correlazioni e gli anni
correlation_df = pd.DataFrame({'Anno': years,'Correlazione': correlations})

# Crea il grafico dell'andamento della correlazione nel tempo
fig = px.line(correlation_df, x='Anno', y='Correlazione',title=f"Correlazione tra {stat1} e {stat2} nel tempo",markers=True)
fig.update_xaxes(tickmode='linear', dtick=1)
fig.show()
